<a href="https://colab.research.google.com/github/akashbilgi/Computer-Network-Lab/blob/main/Part2_from_scratch_DMT_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle


In [ ]:
print("data read")
data = pd.read_csv('/content/drive/MyDrive/data.csv')

data read


In [ ]:
data.drop(data.columns[[-1, 0]], axis=1, inplace=True)

In [ ]:
# convert categorical labels to numbers
diag_map = {'M': 1.0, 'B': -1.0}
data['diagnosis'] = data['diagnosis'].map(diag_map)

In [ ]:
Y = data.loc[:, 'diagnosis']
X = data.iloc[:, 1:]

In [ ]:
def remove_correlated_features(X):
    corr_threshold = 0.9
    corr = X.corr()
    drop_columns = np.full(corr.shape[0], False, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i + 1, corr.shape[0]):
            if corr.iloc[i, j] >= corr_threshold:
                drop_columns[j] = True
    columns_dropped = X.columns[drop_columns]
    X.drop(columns_dropped, axis=1, inplace=True)
    return columns_dropped

In [ ]:
def remove_less_significant_features(X, Y):
    sl = 0.05
    regression_ols = None
    columns_dropped = np.array([])
    for itr in range(0, len(X.columns)):
        regression_ols = sm.OLS(Y, X).fit()
        max_col = regression_ols.pvalues.idxmax()
        max_val = regression_ols.pvalues.max()
        if max_val > sl:
            X.drop(max_col, axis='columns', inplace=True)
            columns_dropped = np.append(columns_dropped, [max_col])
        else:
            break
    regression_ols.summary()
    return columns_dropped

In [ ]:
remove_correlated_features(X)
remove_less_significant_features(X, Y)

array(['smoothness_mean', 'compactness_worst', 'compactness_mean',
       'radius_mean', 'texture_se', 'symmetry_se', 'smoothness_se',
       'concavity_worst'], dtype='<U32')

In [ ]:
    X_normalized = MinMaxScaler().fit_transform(X.values)
    X = pd.DataFrame(X_normalized)

In [ ]:
X.insert(loc=len(X.columns), column='intercept', value=1)

In [ ]:
    print("splitting dataset into train and test sets...")
    X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=42)

splitting dataset into train and test sets...


In [ ]:
def sgd(features, outputs):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)

        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is: {} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights

In [ ]:
def calculate_cost_gradient(W, X_batch, Y_batch):
    # if only one example is passed (eg. in case of SGD)
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])  # gives multidimensional array

    distance = 1 - (Y_batch * np.dot(X_batch, W))
    # print(Y_batch,X_batch,W,distance)
    
    dw = np.zeros(len(W))

    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (regularization_strength * Y_batch[ind] * X_batch[ind])
        dw += di

    dw = dw/len(Y_batch)  # average
    return dw

In [ ]:
def compute_cost(W, X, Y):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = regularization_strength * (np.sum(distances) / N)

    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost

In [ ]:
regularization_strength = 10000
learning_rate = 0.000001

In [ ]:
y_train.to_numpy().shape

(455,)

In [ ]:
    print("training started...")
    W = sgd(X_train.to_numpy(), y_train.to_numpy())
    print("training finished.")
    print("weights are: {}".format(W))

training started...
Epoch is: 1 and Cost is: 7276.643039723538
Epoch is: 2 and Cost is: 6559.6482507901355
Epoch is: 4 and Cost is: 5538.504205841417
Epoch is: 8 and Cost is: 3852.6610406779337
Epoch is: 16 and Cost is: 2661.7617712883675
Epoch is: 32 and Cost is: 2017.9122574703092
Epoch is: 64 and Cost is: 1591.1138288936675
Epoch is: 128 and Cost is: 1329.6797667461324
Epoch is: 256 and Cost is: 1159.882699813013
Epoch is: 512 and Cost is: 1078.066408404689
Epoch is: 1024 and Cost is: 1047.0292453524398
Epoch is: 2048 and Cost is: 1044.597181157041
training finished.
weights are: [ 3.5576233  11.07106087 -2.29600444 -7.87346841 10.14033986 -1.28221655
 -6.44133227  2.2382079  -3.88643032  3.2516378   4.96067164  4.82433492
 -4.70880401]


In [ ]:
print("testing the model...")
y_train_predicted = np.array([])
for i in range(X_train.shape[0]):
    yp = np.sign(np.dot(X_train.to_numpy()[i], W))
    y_train_predicted = np.append(y_train_predicted, yp)

y_test_predicted = np.array([])
for i in range(X_test.shape[0]):
    yp = np.sign(np.dot(X_test.to_numpy()[i], W))
    y_test_predicted = np.append(y_test_predicted, yp)


testing the model...


In [ ]:
print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_test_predicted)))
print("recall on test dataset: {}".format(recall_score(y_test, y_test_predicted)))
print("precision on test dataset: {}".format(recall_score(y_test, y_test_predicted)))


accuracy on test dataset: 0.9824561403508771
recall on test dataset: 0.9534883720930233
precision on test dataset: 0.9534883720930233
